Copyright (c) 2016, EPFL/Blue Brain Project

 This file is part of BluePyOpt <https://github.com/BlueBrain/BluePyOpt>

 This library is free software; you can redistribute it and/or modify it under
 the terms of the GNU Lesser General Public License version 3.0 as published
 by the Free Software Foundation.

 This library is distributed in the hope that it will be useful, but WITHOUT
 ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
 FOR A PARTICULAR PURPOSE.  See the GNU Lesser General Public License for more
 details.

 You should have received a copy of the GNU Lesser General Public License
 along with this library; if not, write to the Free Software Foundation, Inc.,
 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.

# Creating a simple cell optimisation

This notebook will explain how to set up an optimisation of simple single compartmental cell with two free parameters that need to be optimised

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload

First we need to import the module that contains all the functionality to create electrical cell models

In [ ]:
import bluepyopt as nrp
import bluepyopt.electrical as nrpel

If you want to see a lot of information about the internals, 
the verbose level can be set to 'debug' by commenting out
the following lines

In [ ]:
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)

Setting up a cell template
-------------------------
First a template that will describe the cell has to be defined. A template consists of:
* a morphology
* model mechanisms
* model parameters

### Creating a morphology
A morphology can be loaded from a file (SWC or ASC).

In [ ]:
morph = nrpel.morphologies.NrnFileMorphology('simple.swc')

By default a Neuron morphology has the following sectionlists: somatic, axonal, apical and basal. Let's create an object that points to the somatic sectionlist. This object will be used later to specify where mechanisms have to be added etc.

In [ ]:
somatic_loc = nrpel.locations.NrnSeclistLocation('somatic', seclist_name='somatic')

### Creating a mechanism

Now we can add ion channels to this morphology. Let's add the default Neuron Hodgkin-Huxley mechanism to the soma. 

In [ ]:
hh_mech = nrpel.mechanisms.NrnMODMechanism(
        name='hh',
        prefix='hh',
        locations=[somatic_loc])

The 'name' field can be chosen by the user, this name should be unique. The 'prefix' points to the same field in the NMODL file of the channel. 'locations' specifies which sections the mechanism will be added to.

### Creating parameters

Next we need to specify the parameters of the model. A parameter can be in two states: frozen and not-frozen. When a parameter is frozen it has an exact value, otherwise it only has some bounds but the exact value is not known yet.
Let's first a parameter that sets the capitance of the soma to a frozen value

In [ ]:
cm_param = nrpel.parameters.NrnSectionParameter(
        name='cm',
        param_name='cm',
        value=1.0,
        locations=[somatic_loc],
        frozen=True)

And parameters that represent the maximal conductance of the sodium and potassium channels. These two parameters will be optimised later.

In [ ]:
gnabar_param = nrpel.parameters.NrnSectionParameter(                                    
        name='gnabar_hh',
        param_name='gnabar_hh',
        locations=[somatic_loc],
        bounds=[0.05, 0.125],
        frozen=False)     
gkbar_param = nrpel.parameters.NrnSectionParameter(
        name='gkbar_hh',
        param_name='gkbar_hh',
        bounds=[0.01, 0.075],
        locations=[somatic_loc],
        frozen=False)

### Creating the template

To create the cell template, we pass all these objects to the constructor of the template

In [ ]:
simple_cell = nrpel.celltemplate.CellTemplate(
        name='simple_cell',
        morph=morph,
        mechs=[hh_mech],
        params=[cm_param, gnabar_param, gkbar_param])  

Now we can print out a description of the cell

In [ ]:
print simple_cell

With this cell we can build a cell evaluator.

## Setting up a cell evaluator

To optimise the parameters of the cell we need to create cell evaluator object. 
This object will need to know which protocols to injection, which parameters to optimise, etc.

### Creating the protocols

A protocol consists of a set of stimuli, and a set of responses (i.e. recordings). These responses will later be used by a calculate
the score of the parameter values.
Let's create two protocols, two square current pulse at somatic[0](0.5) with different amplitudes.
We first need to create a location object

In [ ]:
soma_loc = nrpel.locations.NrnSeclistCompLocation(
        name='soma',
        seclist_name='somatic',
        sec_index=0,
        comp_x=0.5)


and then the stimuli, recordings and protocols. For each protocol we add a recording and a stimulus in the soma.

In [ ]:
protocols = {}
for protocol_name, amplitude in [('step1', 0.01), ('step2', 0.05)]:
    stim = nrpel.stimuli.NrnSquarePulse(
                step_amplitude=amplitude,
                step_delay=100,
                step_duration=50,
                location=soma_loc,
                total_duration=200)
    rec = nrpel.recordings.CompRecording(
            name='%s.soma.v' % protocol_name,
            location=soma_loc,
            variable='v')
    protocol = nrpel.protocols.Protocol(protocol_name, [stim], [rec])
    protocols[protocol.name] = protocol
    print protocol

### Running protocols on a cell

Now we're at a stage where we can actually run a protocol on the cell. The run_protocols() function of the cell_template accepts a set of protocols, and a dictionary with the values for the parameters that are not frozen.

In [ ]:
default_params = {'gnabar_hh': 0.1, 'gkbar_hh': 0.03}
responses = simple_cell.run_protocols(protocols, param_values=default_params)

Plotting the response traces is now easy:

In [ ]:
def plot_responses(responses):
    plt.subplot(2,1,1)
    plt.plot(responses['step1.soma.v']['time'], responses['step1.soma.v']['voltage'], label='step1')
    plt.legend()
    plt.subplot(2,1,2)
    plt.plot(responses['step2.soma.v']['time'], responses['step2.soma.v']['voltage'], label='step2')
    plt.legend()
    plt.tight_layout()

plot_responses(responses)

As you can see, when we use different parameter values, the response looks different.

In [ ]:
other_params = {'gnabar_hh': 0.12, 'gkbar_hh': 0.05}
plot_responses(simple_cell.run_protocols(protocols, param_values=other_params))

### Defining eFeatures and objectives

For every response we need to define a set of eFeatures we will use for the fitness calculation later. We have to combine features together into objectives that will be used by the optimalisation algorithm. In this case we will create one objective per feature:

In [ ]:
efel_feature_means = {'step1': {'Spikecount': 1}, 'step2': {'Spikecount': 5}}

objectives = []

for protocol_name, protocol in protocols.iteritems():
    stim_start = protocol.stimuli[0].step_delay
    stim_end = stim_start + protocol.stimuli[0].step_duration
    for efel_feature_name, mean in efel_feature_means[protocol_name].iteritems():
        feature_name = '%s.%s' % (protocol_name, efel_feature_name)
        feature = nrpel.efeatures.eFELFeature(
                    feature_name,
                    efel_feature_name=efel_feature_name,
                    recording_names={'': '%s.soma.v' % protocol_name},
                    stim_start=stim_start,
                    stim_end=stim_end,
                    exp_mean=mean,
                    exp_std=0.05 * mean)
        objective = objective = nrpel.objectives.SingletonObjective(
            feature_name,
            feature)
        objectives.append(objective)

### Creating the cell evaluator

We will need an object that can use these objective definitions to calculate the scores from a protocol response. This is called a ScoreCalculator.

In [ ]:
score_calc = nrpel.scorecalculators.ObjectivesScoreCalculator(objectives) 

Combining everything together we have a CellEvaluator. The CellEvaluator constructor has a field 'parameter_names' which contains the (ordered) list of names of the parameters that are used as input (and will be fitted later on).

In [ ]:
cell_evaluator = nrpel.cellevaluator.CellEvaluator(
        cell_template=simple_cell,
        param_names=['gnabar_hh', 'gkbar_hh'],
        fitness_protocols=protocols,
        fitness_calculator=score_calc)


### Evaluating the cell

The cell can now be evaluate for a certain set of parameter values.

In [ ]:
print cell_evaluator.evaluate_with_dicts(default_params)

## Setting up and running an optimisation

Now that we have a cell template and an evaluator for this cell, we can set up an optimisation.

In [ ]:
optimisation = nrp.Optimisation(
        evaluator=cell_evaluator,
        eval_function=cell_evaluator.evaluate_with_lists,
        offspring_size = 10)

And this optimisation can be run for a certain number of generations

In [ ]:
final_pop, hall_of_fame, logs, hist = optimisation.run(max_ngen=5)

The optimisation has return us 4 objects: final population, hall of fame, statistical logs and history. 

The final population contains a list of tuples, with each tuple representing the two parameters of the model

In [ ]:
print 'Final population: ', final_pop

The best individual found during the optimisation is the first individual of the hall of fame

In [ ]:
best_ind = hall_of_fame[0]
print 'Best individual: ', best_ind
print 'Fitness values: ', best_ind.fitness.values

We can evaluate this individual and make use of a convenience function of the cell evaluator to return us a dict of the parameters

In [ ]:
best_ind_dict = cell_evaluator.param_dict(best_ind)
print cell_evaluator.evaluate_with_dicts(best_ind_dict)

As you can see the evaluation returns the same values as the fitness values provided by the optimisation output. 
We can have a look at the responses now.

In [ ]:
responses = simple_cell.run_protocols(protocols, param_values=best_ind_dict)
plot_responses(responses)

Let's have a look at the optimisation statistics.
We can plot the minimal score (sum of all objective scores) found in every optimisation. 
The optimisation algorithm uses negative fitness scores, so we actually have to look at the maximum values log.

In [ ]:
import numpy
gen_numbers = logs.select('gen')
min_fitness = logs.select('min')
max_fitness = logs.select('max')
plt.plot(gen_numbers, min_fitness, label='min fitness')
plt.xlabel('generation #')
plt.ylabel('score (# std)')
plt.legend()
plt.xlim(min(gen_numbers) - 1, max(gen_numbers) + 1) 
plt.ylim(0.9*min(min_fitness), 1.1 * max(min_fitness)) 